In [1]:
import numpy as np
import pandas as pd
import pytrends
import feedparser
from pytrends.request import TrendReq

import sys
from cnn_news_rss import *
from cnn_elastic import *


## http://rss.cnn.com/rss/edition.rss
## http://rss.cnn.com/rss/edition_world.rss

## https://trends.google.com/trends/
## https://www.shanelynn.ie/asynchronous-updates-to-a-webpage-with-flask-and-socket-io/

In [2]:
news_reader = rss_news_reader('http://rss.cnn.com/rss/edition_world.rss')
news_reader.read_news_feed()
news_reader.news.head()

,published,title,link,summary,title_summary,new_flag
0,2018-01-28 16:56:17,Afghanistan mourns after ambulance bomb kills ...,http://rss.cnn.com/~r/rss/edition_world/~3/Ngh...,Afghanistan declared a national day of mournin...,Afghanistan mourns after ambulance bomb kills ...,True
1,2018-01-23 17:06:38,US citizens killed in Kabul hotel last weekend,http://rss.cnn.com/~r/rss/edition_world/~3/7BE...,American citizens were among those killed in t...,US citizens killed in Kabul hotel last weekend...,True
2,2018-01-25 10:52:02,Spokesman for Trump campaign adviser killed in...,http://rss.cnn.com/~r/rss/edition_world/~3/hcG...,"Glenn Selig, spokesman for Trump campaign advi...",Spokesman for Trump campaign adviser killed in...,True
3,2018-01-25 22:09:08,North Korea sanctions are strangling this city,http://rss.cnn.com/~r/rss/edition_world/~3/_R0...,,North Korea sanctions are strangling this city,True
4,2018-01-26 14:23:33,Davos: Bitcoin is not a currency,http://rss.cnn.com/~r/rss/edition_world/~3/67U...,,Davos: Bitcoin is not a currency,True


In [3]:
## load elastic
cnn_es = cnn_es_engine(delete_index = False)


In [4]:
cnn_es

In [19]:
aa = cnn_es.search_query("department korea",val_size=1)
aa['hits']['hits'][0]['_source']['title_summary']

{'_id': 'PtG-PWEB-YHFa6VlzoNA',
 '_index': 'cnn_news',
 '_score': 2.2975755,
 '_source': {'link': 'http://rss.cnn.com/~r/rss/edition_world/~3/vZyvykuUOS0/index.html',
  'published': '2018-01-25T14:24:26',
  'summary': '',
  'title': "South Korea condemns Trump's tariffs",
  'title_summary': "South Korea condemns Trump's tariffs "},
 '_type': 'article'}

In [54]:
es_cnn_res

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'UtG-PWEB-YHFa6Vl0IOx',
    '_index': 'cnn_news',
    '_score': 1.8232571,
    '_source': {'link': 'http://rss.cnn.com/~r/rss/edition_world/~3/MoW7N7xg94c/index.html',
     'published': '2018-01-28T20:09:30.071286',
     'summary': "The scientists and scholars who set the time on the 'Doomsday Clock' do a lot of important things to keep humanity safe.",
     'title': "Running the 'Doomsday Clock' is a full-time job. Really.",
     'title_summary': "Running the 'Doomsday Clock' is a full-time job. Really. The scientists and scholars who set the time on the 'Doomsday Clock' do a lot of important things to keep humanity safe."},
    '_type': 'article'}],
  'max_score': 1.8232571,
  'total': 5},
 'timed_out': False,
 'took': 0}

In [31]:
gg_trends = TrendReq(hl='en-US', tz=360)
tt = gg_trends.trending_searches()
tt['title']

search_res = []

for trend_val in tt['title']:
    ##print (trend_val)
    es_cnn_res = cnn_es.search_query(trend_val,val_size=1)
    try:
        score_val = es_cnn_res['hits']['hits'][0]['_score']
        article_val = es_cnn_res['hits']['hits'][0]['_source']['title_summary']
    except:
        score_val = 0
        article_val = "None"
    search_res.append([trend_val,score_val,article_val])
    
search_res = pd.DataFrame(search_res)
search_res.columns = ['trend','score','article']
search_res

,trend,score,article
0,Grammys 2018,1.776660,"Books of the Week 01/28/2018 ""Twilight of the ..."
1,Roger Federer,2.961390,Day 12: Roger Federer beats Hyeon Chung in se...
2,Jay Z,0.000000,None
3,Caroline Wozniacki,0.000000,None
4,Pro Bowl 2018,1.776660,"Books of the Week 01/28/2018 ""Twilight of the ..."
5,Senior Bowl,1.254523,Nearly 11 months into the Trump administratio...
6,Scott Baio,0.000000,None
7,Nhl All Star Game,0.000000,None
8,Duke basketball,0.000000,None
9,Saturday Night Live,1.686077,See highlights from CNN Heroes tribute CNN's A...


In [33]:
re.sub("\+"," ",re.search("q=([^&]*)",tt['exploreUrl'][14]).group(1))


'Andre Roberson'

In [53]:
from datetime import datetime
datetime.now()+5


TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'int'